In [5]:
import pandas as pd

In [6]:
print("Hello World")

Hello World


In [8]:
a = pd.read_csv("transactions.csv")
print(a)
low_memory=False

                             transaction_id            timestamp     amount  \
0      f2f3b0fc-b7d7-4d85-b682-cbf07ef77c1a  2025-03-06 12:04:00    7746.03   
1      feccbd1e-7752-4452-a44a-901510252561  2025-03-31 01:37:00   45193.04   
2      bc6a480c-e2e1-434d-830c-1cda3f913038  2025-04-29 22:58:00   33029.71   
3      b832aa2c-77f3-4c4f-9697-9776b8d74abb  2025-02-27 14:52:00  35994 .61   
4      0d32c6ef-508e-4f5d-bc64-341a0a9f498d  2025-01-03 22:29:00   32229.73   
...                                     ...                  ...        ...   
99995  ea7e0cb2-fadb-496c-962b-7e54a5cfa1bc  2025-02-03 17:00:00     146.04   
99996  4271c3c0-d3be-49f5-8154-ee60e2a1e582  2025-01-07 14:00:00      56.98   
99997  23574238-9739-4af4-a823-5b699812db4d  2025-05-11 13:00:00     128.34   
99998  38e785ff-e460-419e-af07-595f5c67aaf6  2025-02-02 13:00:00     149.13   
99999  94e5a276-d5ce-40f6-b17c-97885b455bbd  2025-05-09 15:00:00      41.77   

      currency            sender_account          r

C:\Users\Max\AppData\Local\Temp\ipykernel_7636\994900515.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv("transactions.csv")
